In [ ]:
import requests
import pandas as pd
from IPython.display import display, HTML, FileLink
import json
import datetime

BASE_URL = "https://api.ransomware.live/v2"

# ------- functions --------

def api_get(endpoint: str, params=None):
    url = BASE_URL + endpoint
    try:
        resp = requests.get(url, params=params, timeout=15)
        resp.raise_for_status()
        return resp.json()
    except requests.RequestException as e:
        print(f"[ERROR] {endpoint}: {e}")
        return None

def show_scrollable_df(df: pd.DataFrame, height_px=350, width_px=1200, max_rows=1000, max_cols=100):
    if df.empty:
        print("No data found or error occurred.")
    else:
        nrows, ncols = df.shape
        shown_rows = min(nrows, max_rows)
        shown_cols = min(ncols, max_cols)
        print(f"Showing {shown_rows} of {nrows} rows, {shown_cols} of {ncols} columns")
        styles = f"""
        <div style="overflow-x:auto; overflow-y:auto; height:{height_px}px; width:{width_px}px; border:1px solid #ccc;">
        {df.iloc[:max_rows, :max_cols].to_html(max_rows=max_rows, max_cols=max_cols, escape=False)}
        </div>
        """
        display(HTML(styles))

def ask_save_csv(df: pd.DataFrame, name=None):
    if name is None:
        name = f"output_{datetime.datetime.now():%Y%m%d_%H%M%S}.csv"
    save = input("Do you want to save the result as a CSV file? (y/n): ").strip().lower()
    if save == 'y':
        df.to_csv(name, index=False)
        print(f"Saved as {name}")
        display(FileLink(name))
    else:
        show_scrollable_df(df)

def json_to_csv(data, filename):
    if isinstance(data, list) and len(data) and isinstance(data[0], dict):
        df = pd.json_normalize(data)
        ask_save_csv(df, filename)
    elif isinstance(data, dict):
        try:
            df = pd.json_normalize(data)
            ask_save_csv(df, filename)
        except Exception as e:
            print("[!] Could not convert JSON to CSV:", e)
            print(json.dumps(data, indent=2))
    else:
        print("[!] JSON structure is not tabular; cannot convert to CSV.")
        print(json.dumps(data, indent=2))

# -------- API functions used ---------

def get_groups():
    data = api_get("/groups")
    if data:
        json_to_csv(data, "ransomware_groups.csv")

def get_recent_victims():
    data = api_get("/recentvictims")
    if data:
        json_to_csv(data, "recent_victims.csv")

def get_group_details():
    group = input("Enter ransomware group name (e.g., lockbit): ").strip().lower()
    data = api_get(f"/group/{group}")
    if data:
        json_to_csv(data, f"{group}_group_details.csv")

def get_all_cyberattacks():
    data = api_get("/allcyberattacks")
    if data:
        json_to_csv(data, "all_cyberattacks.csv")

def get_recent_cyberattacks():
    data = api_get("/recentcyberattacks")
    if data:
        json_to_csv(data, "recent_cyberattacks.csv")

def get_groupvictims():
    group = input("Enter ransomware group name: ").strip().lower()
    data = api_get(f"/groupvictims/{group}")
    if data:
        json_to_csv(data, f"{group}_victims.csv")

def search_victims():
    keyword = input("Enter search keyword: ").strip()
    data = api_get(f"/searchvictims/{keyword}")
    if data:
        json_to_csv(data, f"victims_search_{keyword}.csv")

def victims_by_country():
    country = input("Enter country code (ISO2, e.g., IN, US): ").strip().upper()
    data = api_get(f"/countryvictims/{country}")
    if data:
        json_to_csv(data, f"{country}_victims.csv")

def victims_by_month():
    year = input("Year (e.g., 2024): ").strip()
    month = input("Month (1-12): ").strip().zfill(2)
    data = api_get(f"/victims/{year}/{month}")
    if data:
        json_to_csv(data, f"victims_{year}_{month}.csv")
    else:
        print(f"No ransomware victim data found for {year}-{month}.")

def victims_by_sector():
    sector = input("Sector (e.g., healthcare): ").strip().lower()
    data = api_get(f"/sectorvictims/{sector}")
    if data:
        json_to_csv(data, f"{sector}_sector_victims.csv")

def victims_by_sector_and_country():
    sector = input("Sector (e.g., education): ").strip().lower()
    country = input("Country code (ISO2): ").strip().upper()
    data = api_get(f"/sectorvictims/{sector}/{country}")
    if data:
        json_to_csv(data, f"{sector}_{country}_sector_victims.csv")

def get_cert():
    country = input("Country code (ISO2): ").strip().upper()
    data = api_get(f"/certs/{country}")
    if data:
        print(json.dumps(data, indent=2))
    else:
        print("No CERT data found.")

def get_yara():
    group = input("Ransomware group: ").strip()
    api_url = f"https://api.ransomware.live/v2/yara/{group}"
    resp = requests.get(api_url)
    try:
        data = resp.json()
        if isinstance(data, dict) and 'rule' in data and data['rule'].strip():
            rule = data['rule']
            print("\nYARA Rule:\n")
            print(rule)
            save = input("Do you want to save this YARA rule as a .yar file? (y/n): ").strip().lower()
            if save == 'y':
                fname = f"{group}.yar"
                with open(fname, "w", encoding="utf-8") as f:
                    f.write(rule)
                print(f"Saved as {fname}")
        else:
            print(f"No YARA rule found for '{group}'. Full API response:")
            print(json.dumps(data, indent=2))
    except Exception as e:
        print("Error decoding JSON from API:", e)
        print(resp.text)

# -------- CLI --------

def main_menu():
    menu = [
        ("List All Ransomware Groups", get_groups),
        ("Recent Victims", get_recent_victims),
        ("All Known Cyberattacks", get_all_cyberattacks),
        ("Recent Cyberattacks", get_recent_cyberattacks),
        ("Victims by Group", get_groupvictims),
        ("Victims by Keyword", search_victims),
        ("Victims by Country", victims_by_country),
        ("Victims by Year/Month", victims_by_month),
        ("Victims by Sector", victims_by_sector),
        ("Victims by Sector+Country", victims_by_sector_and_country),
        ("CERT by Country", get_cert),
        ("YARA Rules for Group", get_yara),
        ("Group Details", get_group_details),
        ("Exit", None)
    ]
    while True:
        print("\n" + "="*36)
        print(" Ransomware.live Data Fetch Menu")
        print("="*36)
        for i, (desc, _) in enumerate(menu, 1):
            print(f"{i}. {desc}")
        try:
            choice = int(input(f"\nSelect option (1-{len(menu)}): ").strip())
            if 1 <= choice < len(menu):
                menu[choice-1][1]()
            elif choice == len(menu):
                print("Goodbye!")
                break
            else:
                print("Invalid selection.")
        except Exception as e:
            print("Invalid input or error occurred:", e)

main_menu()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.display import display, HTML

def fetch_iocs_by_group():
    resp = requests.get("https://www.ransomware.live/ioc", timeout=10)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    groups = {}
  
    for table in soup.select("table"):
       
        cap = table.find("caption")
        if cap:
            group = cap.text.strip()
        else:
           
            prev = table.find_previous(lambda tag: tag.name in ["h2", "h3", "p"])
            group = prev.text.strip() if prev else "unknown"
        
        buf = StringIO(str(table))
        df = pd.read_html(buf, displayed_only=False)[0]
        if group in groups:
            groups[group] = pd.concat([groups[group], df], ignore_index=True)
        else:
            groups[group] = df

    return groups


groups = fetch_iocs_by_group()
print(f"Found IOCs for {len(groups)} ransomware families")
for g, df in groups.items():
    print(f"\n=== {g} ({len(df)} IOCs) ===")
    display(HTML(df.to_html(index=False)))
    df.to_csv(f"iocs_{g}.csv", index=False)
    print(f"Saved iocs_{g}.csv")
